In [ ]:
import requests
import pandas as pd
import numpy as np

In [ ]:
def divide_chunks(n, d=100):
    return int((n+ (d-1))/d)

In [ ]:
base_url = "https://api.fabdb.net/cards"

payload = {
    "page": 1,
    "per_page": 100,
}

pages = []
for i in range(1, divide_chunks(656, 100)):
    r = requests.get(base_url, params=payload)
    pages.append(r.json())
    payload['page'] += 1

In [ ]:
dfs = []
for i in pages:
    dfs.append(pd.DataFrame.from_dict(i['data']))
df = pd.concat(dfs).reset_index()

In [ ]:
stat_list = df.stats.values.tolist()
for n, i in enumerate(stat_list):
    if i == []:
        stat_list[n] = {}
        
df = df.drop(['stats', 'index'], axis=1).join(pd.DataFrame(stat_list))
# numeric_cols = ['resource', 'attack', 'cost', 'defense', 'intellect', 'life']
# df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
def transform_name(row):

    resource_map = {
        '3': '(Blue)',
        '2': '(Yellow)',
        '1': '(Red)',
        np.nan: '',
    }
    
    return f"{row['name']} {resource_map[row['resource']]}"

In [ ]:
df['name_resource'] = df.apply(transform_name, axis=1)

In [ ]:
df = df[~df['identifier'].str.contains("IRA")]
df = df.drop_duplicates('name_resource')


In [ ]:
df.to_csv("cards.csv", index=False)

In [ ]:
df[df['name'] == "Harmonized Kodachi"]